In [ ]:
import json
import sys
import collections
import copy

In [ ]:
# import re
# import pandas as pd

# ans_pattern = re.compile(r'@content\d@')
# def is_multi_answer(answer):
#     docs = ans_pattern.findall(answer)
#     return len(set([int(doc[-2:-1]) for doc in docs])) > 1

# train_df = pd.read_csv('../input/original/train_round_0.csv')
# train_df['is_multi_answer'] = train_df['answer'].map(is_multi_answer)

In [ ]:
# from eval_metric import normalize, compute_bleu_rouge

## 规则方法得到多答案问题id

In [ ]:
question_dict = {}
with open('/home/lq/Research/Reading-Comprehension/les-military-mrc/input/mrc_dataset_0912/test_r0.json') as f:
    for line in f:
        sample = json.loads(line)
        question_dict[sample['question_id']] = sample['question']

In [ ]:
len(question_dict)

In [ ]:
cnt = 0
multi_ans_question_id_list = []
for id_, question in question_dict.items():
    if (question.count('?') + question.count('？')) > 1 or (question.count('多少') + question.count('哪') + question.count('什么') + question.count('谁')) > 1:
        cnt += 1
        multi_ans_question_id_list.append(id_)
#     elif '分别' in question and '和' in question:
#         print(id_, question)
#     elif '哪两' in question:
#         print(id_, question)
    
print(cnt)

In [ ]:
multi_ans_question_id_list += ['Q_06031cjhhh', 'Q_47760knsdc', 'Q_52500cnjsq', 'Q_36961chjcs', 'Q_22570hycqj',
                               'Q_40246dnjsn']

In [ ]:
len(multi_ans_question_id_list)

## bert classifier模型得到多答案id

In [ ]:
is_multi_list = []
with open('../les_mrc/multi_ans_classifier/models/multi_ans_classifier/pred_results.txt') as f:
    for line in f:
        is_multi_list.append(int(line.strip()))
print(len(is_multi_list))

In [ ]:
multi_ans_new = []
with open('/home/lq/Research/Reading-Comprehension/les-military-mrc/input/mrc_dataset_0912/test_r0.json') as f:
    for i, line in enumerate(f):
        sample = json.loads(line)
        if is_multi_list[i] == 1:
            multi_ans_new.append(sample['question_id'])
            if sample['question_id'] not in multi_ans_question_id_list:
#                 print(sample['question'])
                pass
print(len(multi_ans_new))

In [ ]:
multi_ans_new += ['Q_06031cjhhh', 'Q_47760knsdc', 'Q_52500cnjsq', 'Q_36961chjcs', 'Q_22570hycqj',
                               'Q_40246dnjsn']
multi_ans_new = list(set(multi_ans_new))
print(len(multi_ans_new))

## 生成多答案

In [ ]:
with open('nbest_predictions_checkpoint_test.json') as f:
    all_nbest_json = json.loads(f.read())

In [ ]:
all_samples = collections.defaultdict(list)
need_skippeed_list = [',', '.', '。']
for qas_id, nbest_json in all_nbest_json.items():
    text = ''
    prob = 0.0
    logit = 0.0
    for entry in nbest_json:
        if entry['text'].strip() and entry['text'] not in need_skippeed_list:
            text = entry['text'].strip()
            logit = entry['start_logit'] + entry['end_logit']
            prob = entry['probability']
            break
    all_samples[qas_id.split('##')[0]].append([text, logit, prob])
all_predictions = {}
multi_pred = []
cnt = 0
for question_id, sample in all_samples.items():
    sample = sorted(sample, key=lambda x: x[1], reverse=True)
    all_predictions[question_id] = sample[0][0]
    # 简单的多答案选择模块
    if question_id in multi_ans_new:
#         for i in range(1, 5):
#             ans1 = normalize([sample[0][0]])
#             ans2 = normalize([sample[i][0]])
#         if sample[0][0] == sample[1][0]:
#             continue
        all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        print(all_predictions[question_id])
        cnt += 1
#     if sample[1][0] != '' and sample[1][2] > 0.5:
#         # 有可能具有多答案
#         if sample[0][0] == sample[1][0]:
#             continue
#         if sample[1][0] in sample[0][0] or sample[0][0] in sample[1][0]:
#             continue
#         ans1 = normalize([sample[0][0]])
#         ans2 = normalize([sample[1][0]])
#         bleu_rouge = compute_bleu_rouge({'item': ans1}, {'item': ans2})
#         if bleu_rouge['Bleu-4'] > 0.2 or bleu_rouge['Rouge-L'] > 0.2:
#             continue
#         print('{} have multi-ans, take care of it'.format(question_id))
#         multi_pred.append(question_id)
#         cnt += 1
#         all_predictions[question_id] = all_predictions[question_id] + '#' + sample[1][0]
        
print(cnt)

In [ ]:
with open('multi_ans_0921_test.json', "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False) + "\n")